### **🧠 머신러닝 실습 - 이진 분류**
#### Q1. 환자의 당뇨병 여부를 예측하시오.
- 제공된 데이터 : `diabetes_train.csv`, `diabetese_test.csv`
- 예측할 컬럼 : `Outcome` (0: 정상, 1: 당뇨병)


학습용 데이터(train)을 이용해 환자의 당뇨병을 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값 (당뇨병일 확률)
- 제출 파일명 : `new.csv`


제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점한다.

In [43]:
import pandas as pd

## 데이터 불러오기
train = pd.read_csv('diabetes_train.csv')
test = pd.read_csv('diabetes_test.csv')

## EDA
# print(train.shape, test.shape) # ((614, 9), (154, 8))
# train.info() # not object
# test.info()
# train.describe()
# print(train.isnull().sum())
# print(test.isnull().sum())
# print(train['Outcome'].value_counts()) # 0: 403 / 1: 211

## 데이터 전처리
y_train = train.pop('Outcome')
## - 스케일링
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
train = minmax.fit_transform(train)
test = minmax.transform(test)


## 검증 데이터 나누기
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=42)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape) # (491, 8) (123, 8) (491,) (123,)

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestClassifier

# 하이퍼파라미터 튜닝
rf = RandomForestClassifier(max_depth=5, n_estimators=500, random_state=0)
# rf = RandomForestClassifier(random_state=0) # 기본

rf.fit(X_train, y_train)
pred = rf.predict_proba(X_val)
# print(rf.classes_)
# print(pred)

from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

## 예측 및 결과 파일 생성
pred = rf.predict_proba(test)
result = pd.DataFrame({'pred':pred[:,1]})
result.to_csv('new.csv', index=False)

pd.read_csv('new.csv').shape


roc_auc: 0.8234969663541093


(154, 1)

스케일링 전 roc_auc : 0.8167402095973524



스케일링 후 roc_auc : 0.8234969663541093


**[하이퍼파라미터 튜닝]**
- `max_depth` : 트리의 최대 깊이, 과적합 방지, 3~7 정도 적당
- `n_estimators` : 트리의 개수, 높을수록 안정적 예측 가능(학습 시간 소요), 200~500 정도 적당

---

#### Q2. 새로운 일자리를 찾을지 예측하시오.
- 제공된 데이터 : `hr_train.csv`, `hr_test.csv`
- 예측할 컬럼 : `target` (0: 새로운 일자리를 찾지 않음, 1: 새로운 일자리를 찾음)


학습용 데이터(train)을 이용해 새 일자리를 찾을지 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값 (이직할 확률)
- 제출 파일명 : `new2.csv`


제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점한다.

In [52]:
## 라이브러리 및 데이터 불러오기
import pandas as pd

train = pd.read_csv('hr_train.csv')
test = pd.read_csv('hr_test.csv')

## EDA
# print(train.info())
# print(test.info())
# print(train.isnull().sum()) # gender, enrolled_university, education_level, major_discipline, experience, company_size, company_type, last_new_job
# print(test.isnull().sum())
# train.describe()
# train.describe(include='O')
# test.describe(include='O')
# train['target'].value_counts() # 0.0: 11517 / 1.0: 3809
# print(train.shape, test.shape) # (15326, 14) (3832, 13)

## 데이터 전처리
# train = train.drop('enrollee_id', axis=1)
# test = test.drop('enrollee_id', axis=1)

train = train.fillna('X')
test = test.fillna('X')

# print(train.isnull().sum().sum(), test.isnull().sum().sum())

y_train = train.pop('target')

# 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)
# train.shape, test.shape # 불일치

df = pd.concat([train, test], axis=0)
df_oh = pd.get_dummies(df)
train = df_oh.iloc[:len(train)].copy()
test = df_oh.iloc[len(train):].copy()

# train.shape, test.shape # 일치

# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

cols_obj = train.select_dtypes(include='O').columns

for col in cols_obj :
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

train = df[:len(train)].copy()
test = df[len(train):].copy()

# 스케일링
from sklearn.preprocessing import MinMaxScaler

cols_num = train.select_dtypes(exclude='O').columns

minmax = MinMaxScaler()
train[cols_num] = minmax.fit_transform(train[cols_num])
test[cols_num] = minmax.transform(test[cols_num])

## 검증 데이터 나누기
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train,
                                                  y_train,
                                                  test_size=0.2, 
                                                  random_state=42)

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0, max_depth=5, n_estimators=500)
rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_val)

# print(rf.classes_)
# print(y_pred)

from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_val, y_pred[:,1])
print('roc-auc:', roc_auc)

## 예측 및 결과 파일 생성
pred = rf.predict_proba(test)
result = pd.DataFrame({'pred':pred[:,1]})
result.to_csv('new2.csv', index=False)

pd.read_csv('new2.csv').head()




roc-auc: 0.7952435945564116


,pred
0,0.234171
1,0.394253
2,0.411359
3,0.140237
4,0.157046


In [ ]:
cols = train.select_dtypes(include='O')

for col in cols:
    train_set = set(train[col])
    test_set = set(test[col])

    if train_set == test_set :
        print(col, '동일')
    else :
        print(col, '동일하지 않음')

---
#### Q3. 신용카드 신청자의 채무 불이행을 예측하시오.
- 제공된 데이터 : `creditcard_train.csv`, `creditcard_test.csv`
- 예측할 컬럼 : `STATUS` (0: 채무 이행, 1: 채무 불이행)


학습용 데이터(train)을 이용해 미래의 채무 불이행을 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.


제출 파일은 다음 1개의 컬럼을 포함해야 한다.
- pred : 예측값 (채무 불이행 확률)
- 제출 파일명 : `new3.csv`


제출한 모델의 성능은 F1 평가지표에 따라 채점한다.

In [83]:
import pandas as pd

train = pd.read_csv('creditcard_train.csv')
test = pd.read_csv('creditcard_test.csv')

cols_obj = train.select_dtypes(include='O').columns

for col in cols_obj :
    train_set = set(train[col])
    test_set = set(test[col])

    if train_set == test_set :
        print(col, '동일')
    else :
        print(col, '동일하지 않음')

print('-----------------------')

## EDA
# train.info()
# test.info()
# train.shape, test.shape # ((25519, 19), (7591, 18))
# train.isnull().sum() # OCCUPATION_TYPE 
# test.isnull().sum()
# train['STATUS'].value_counts() # 0: 25085 / 1: 434

## 데이터 전처리
# train['OCCUPATION_TYPE'].value_counts()
train['OCCUPATION_TYPE'] = train['OCCUPATION_TYPE'].fillna(train['OCCUPATION_TYPE'].mode()[0])

train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)

cols_obj = train.select_dtypes(include='O').columns

y_train = train.pop('STATUS')

# # 원-핫 인코딩
# train = pd.get_dummies(train)
# test = pd.get_dummies(test)
# # train.shape, test.shape

# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

for col in cols_obj :
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
    
# # 스케일링 (개선 x)
# from sklearn.preprocessing import MinMaxScaler

# cols_num = train.select_dtypes(exclude='O').columns

# minmax = MinMaxScaler()
# train[cols_num] = minmax.fit_transform(train[cols_num])
# test[cols_num] = minmax.transform(test[cols_num])

## 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, 
                                                  y_train, 
                                                  test_size=0.2, 
                                                  random_state=0)

## 모델 학습 및 평가
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0, n_estimators=500, class_weight='balanced')
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
print(rf.classes_)
print(y_pred)

from sklearn.metrics import f1_score

f1 = f1_score(y_val, y_pred)
print('f1-score:', f1)

## 예측 및 결과 파일 생성
pred = rf.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv('new3.csv', index=False)

pd.read_csv('new3.csv').shape


CODE_GENDER 동일
FLAG_OWN_CAR 동일
FLAG_OWN_REALTY 동일
NAME_INCOME_TYPE 동일
NAME_EDUCATION_TYPE 동일
NAME_FAMILY_STATUS 동일
NAME_HOUSING_TYPE 동일
OCCUPATION_TYPE 동일하지 않음
-----------------------
[0 1]
[0 0 0 ... 0 0 0]
f1-score: 0.3241106719367589


(7591, 1)